# 지도학습(4) - 회귀모델 성능 측정 지표

회귀분석을 통해 예측 모델을 만들고 해당 모델의 성능을 파악하기 위해 제동되는 사이킷런의 성능 측정 지표 모듈

## #01. 작업준비

### 패키지 가져오기

In [2]:
import sys

sys.path.append("../../")
import helper

import numpy as np
import seaborn as sb
from pandas import read_excel, DataFrame
from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

# 성능 측정 지표 모듈 (실제값과 예측값 측정 = 잔차를 통해 성능을 측정하는 것)
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error

## #02. 자동차 속도에 따른 제동거리 회귀모델(샘플 회귀분석 모델 만들기)
### 데이터 가져오기

In [3]:
origin = read_excel("https://data.hossam.kr/E04/cars.xlsx")
origin.head()

,speed,dist
0,4,2
1,4,10
2,7,4
3,7,22
4,8,16


### 독립변수에 대한 다항식 생성

In [4]:
poly = PolynomialFeatures(include_bias = False) #1을 포함하지 않는 다항식 객체 생성
fit = poly.fit_transform(origin[['speed']]) # 훈련
x = DataFrame(fit, columns=poly.get_feature_names_out()) # 데이터프레임으로 
x.head()

,speed,speed^2
0,4.0,16.0
1,4.0,16.0
2,7.0,49.0
3,7.0,49.0
4,8.0,64.0


### 종속변수 추출

In [5]:
y = origin[['dist']] # 데이터프레임
y.head()

,dist
0,2
1,10
2,4
3,22
4,16


### 데이터 분할

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=245) 
x_train.shape, x_test.shape, y_train.shape, y_test.shape 

((35, 2), (15, 2), (35, 1), (15, 1))

### 회귀분석 수행

In [7]:
# 객체 생성
model = LinearRegression()

# 훈련시키기
fit = model.fit(x_train, y_train)

print("계수: ", fit.coef_)
print("절편: ", fit.intercept_)
print("훈련데이터 설명력: ", fit.score(x_train, y_train))
print("검증데이터 설명력: ", fit.score(x_test, y_test))

계수:  [[2.66217443 0.01877458]]
절편:  [-5.59048856]
훈련데이터 설명력:  0.6369324456365243
검증데이터 설명력:  0.6033674309812221


### 예측값 생성

#### 훈련데이터에 대한 예측값

In [8]:
y_train_pred = fit.predict(x_train)
y_train_pred = y_train_pred.reshape(-1) #차수 감소시키기
y_train_pred

# 차수를 1 감소시켜라 (squeez()쓰먼 된다.)
#y_train_pred = fit.predict(x_train)
#y_train_pred.squeeze()

array([32.19068386, 45.09233173, 72.69798759, 48.41161662, 48.41161662,
       55.16283391, 41.81059601, 25.96515493, 38.56640946, 45.09233173,
       22.90871422, 35.35977207, 35.35977207, 55.16283391, 29.05914481,
       35.35977207, 51.76845068, 29.05914481, 19.88982269,  5.35860252,
       51.76845068, 48.41161662, 55.16283391, 29.05914481, 13.96468711,
       45.09233173, 25.96515493, 22.90871422, 32.19068386, 29.05914481,
       55.16283391, 51.76845068, 55.16283391, 65.57127861, 62.06424788])

#### 검증데이터에 대한 예측값

In [10]:
y_test_pred = fit.predict(x_test)
y_test_pred = y_test_pred.reshape(-1)
y_test_pred

array([69.11585852,  5.35860252, 32.19068386, 69.11585852, 22.90871422,
       35.35977207, 41.81059601, 16.90848032, 69.11585852, 38.56640946,
       13.96468711, 32.19068386, 38.56640946, 69.11585852, 48.41161662])

## #03. 회귀분석 모델의 성능 평가

회귀분석 모델의 평가를 위한 지표는 실제값(관측치)과 회귀 예측값의 차이를 기반으로 한다.

| 구분 | 설명 |
|---|---|
| 에러율 | 낮을수록 좋음 (0에 가까울 수록 좋음) |
| 설명력 | 높을수록 좋음 (1에 가까울 수록 좋음) |

### 1) 설명력
#### $ R^2 $ (결정계수)

회귀분석에서 가장 많이 채택되는 설명력 값

기본적으로 모델의 학습 결과를 갖고 있는 `fit` 객체의 `score()` 메서드를 통해서 조회 가능

In [11]:
print("훈련 데이터 설명력:", fit.score(x_train, y_train))
print("검증 데이터 설명력:", fit.score(x_test, y_test))

훈련 데이터 설명력: 0.6369324456365243
검증 데이터 설명력: 0.6033674309812221


fit.score()는 학습을 통해 성능판단 <-> r2_score()는 잔차를 이용해 성능 판단

sklearn이 제공하는 `metrics객체`의 `r2_score()`메서드를 통해서도 조회할 수 있다.

이때 파라미터는 관측치와 예측치를 전달한다.

In [12]:
print("훈련 데이터 설명력:", r2_score(y_train, y_train_pred))
print("검증 데이터 설명력:", r2_score(y_test, y_test_pred))

훈련 데이터 설명력: 0.6369324456365243
검증 데이터 설명력: 0.6033674309812221


### 2) 에러율

| 종류 | 이름 | 한글명 | 잔차계산 | 이상치에 영향 여부 |
|----|----|----|----|----|
| MAE | Mean Absoulte Error | 평균절대오차 | 절대값 | YES |
| MSE | Mean Squared Error | 평균제곱오차 | 제곱값 | NO |
| RMSE | Root Mean Squared Error | 평균오차 | 제곱값 | NO |
| MAPE | Mean Absolute Percentage Error | 평균 절대 백분 오차 비율 | 절대값 | YES |
| MPE | Mean Percentage Error | 평균 비율 오차 | N/A | YES |

#### $MAE$ (Mean Absolute Error) : 평균 절대 오차

![mae_expr](./res/mae_expr.png)

모델의 예측값과 실제값의 차이 (=오차) 를 모두 더하는 개념이다. (-> MAE값은 작아야한다. )

절대값을 취하기 때문에 직관적으로 알 수 있는 지표다.

이상치에 영향을 미치지 않는다.

MAE는 절대값을 취하는 지표이기 때문에 실제보다 낮은 값(umderperformance)인지 큰(overperformance) 값인지 알 수 있다.

![mae](./res/mae.jpg)


In [13]:
print("훈련데이터 MAE:", mean_absolute_error(y_train, y_train_pred))
print("검증데이터 MAE:", mean_absolute_error(y_test, y_test_pred))

훈련데이터 MAE: 9.303627957215797
검증데이터 MAE: 16.00879141924198


#### $ MSE $ (Mean Squared Error) : 평균 제곱 오차

![mse](./res/mse_expr.png)

MAE와는 다르게 제곱을 하기 때문에 모델의 실제값과 예측값의 차이의 면적의 합이다.

제곱을 하기 때문에 특이값이 존재하면 수치가 많이 늘어난다 (=특이치에 민감함)

![mse](./res/mse.jpg)

In [14]:
print("훈련데이터 MSE:", mean_squared_error(y_train, y_train_pred))
print("검증데이터 MSE:", mean_squared_error(y_test, y_test_pred))

훈련데이터 MSE: 130.10977792672625
검증데이터 MSE: 509.11051434683174


#### $ RMSE $ (Root MEan Squared Error) : 평균 오차

![RMSE](./res/rmse_expr.png)

MSE를 구한 값에 루트를 씌운다.

오류 지표를 실제 값과 유사한 단위로 변환하여 해석을 쉽게 한다. 

In [15]:
print("훈련데이터 RMSE:", np.sqrt(mean_squared_error(y_train, y_train_pred)))
print("검증데이터 RMSE:", np.sqrt(mean_squared_error(y_test, y_test_pred)))

훈련데이터 RMSE: 11.406567315661897
검증데이터 RMSE: 22.563477443577526


#### $ MAPE $ (Mean Absolute Percentage Error) : 평균 절대 백분오차 비율

![mape](./res/mape_expr.png)

MAE를 퍼센트로 변환한 것이다.

MAE와 동일하게 MSE보다 이상치에 민감하며 실제값보다 낮은 값인지 높은 값인지 알 수 없다.

모델에 대한 편향이 있다.(이를 대응하기 위해 MPE도 추가로 확인하는 것을 추천)

![mape](./res/mape.jpg)

In [16]:
# API로 제공되는 기능이 아니고, 직접 계산해야 하기 때문에 관측치와 예측치의 데이터 타입이 일치해야 한다.
# -> numpy 배열 혹은 Series 타입으로 통일해야 한다.
print("훈련데이터 MAPE:", np.mean(np.abs((y_train.values - y_train_pred)/y_train.values))*100)
print("검증데이터 MAPE:", np.mean(np.abs((y_test.values - y_test_pred)/y_test.values))*100)

훈련데이터 MAPE: 31.953140481807633
검증데이터 MAPE: 146.53926751162095


#### $ MPE $ (Mean Percentage Error) : 평균 비율 오차

![mpe](./res/mpe_expr.png)

MAPE와 비슷하지만 MAPE에서 절대값을 제외한 지표다.

장점은 모델이 실제값보다 낮은 값인지 큰 값인지 판단 할 수 있다.

![mpe](./res/mpe.jpg)


In [17]:
# API로 제공되는 기능이 아니고, 직접 계산해야 하기 때문에 관측치와 예측치의 데이터 타입이 일치해야 한다.
# -> numpy 배열 혹은 Series 타입으로 통일해야 한다.
print("훈련데이터 MAPE:", np.mean((y_train.values - y_train_pred)/y_train.values)*100)
print("검증데이터 MAPE:", np.mean((y_test.values - y_test_pred)/y_test.values)*100)

훈련데이터 MAPE: -13.990507735807734
검증데이터 MAPE: -92.33806169557141
